# Optimization: Assignment Problem

The aim of this notebook is to follow the process of solving a standard assignment problem, which may be performed using different optimization approaches, namely:

- Minimum-Cost Flow
- Mixed Integer Programming
- Constraint Pogramming

For the sake of this example, MIP will be used despite its low time-efficiency vs MCF.


Created by *Sebastián Arango*

### Problem Formulation

In [1]:
!pip install --upgrade --user ortools

Requirement already up-to-date: ortools in /root/.local/lib/python3.6/site-packages (7.6.7691)


In [0]:
import numpy as np
import pandas as pd

from google.colab import files
from datetime import datetime
from __future__ import print_function
from ortools.linear_solver import pywraplp

**Problem statement:**

Having, for instance, a set of tasks $T$ to be performed by a group of workers $W$, the problem is to assign each worker $w$ to a distinct task $t$ so as to minimize the total cost of performance. In order to generalize the problem, workers can be grouped into teams $G$ with tasks distributed among them.

**Objetive Function**

$$\min \ TotalCost = \sum_{w \in W} \sum_{t \in T} Cost_{wt} \cdot Assign_{wt} $$

**Constraint:** Each worker $ w $ is assigned to at most one task $t$.

$$\sum_{t \in T} Assign_{wt} \le 1, \forall w$$

**Constraint:** Each task $t$ is assigned to exactly one worker $w$.

$$\sum_{w \in W} Assign_{wt} = 1, \forall t$$



**Constraint:** All tasks $t$ are equally distributed among each team $g \in G$.

$$\sum_{w \in g} Assign_{wt} = \frac { \sum_{t \in T} t } { \sum_{g \in G} g }, \forall g \in G$$

## Mixed-Integer Programming

In [3]:
# Define a cost matrix with w workers and t tasks

num_workers = 10
num_tasks = 8
num_teams = 2

team_1 = [w for w in range(num_workers) if w%2 != 0] # [1, 3, 5, 7, 9]
team_2 = [w for w in range(num_workers) if w%2 == 0] # [0, 2, 4, 6, 8]

# Create random cost matrix
cost_matrix = np.random.randint(100, size=(num_workers, num_tasks))

cost_matrix

array([[63, 79, 56, 52, 61, 64, 32, 11],
       [84, 62, 77, 21, 82, 70, 13, 30],
       [79, 22, 18, 65, 59, 14, 91, 86],
       [ 2, 35, 44, 17, 63, 77, 84, 47],
       [73, 29, 19, 94, 34, 93, 64, 91],
       [59, 16, 40, 79, 10, 17, 41, 95],
       [60, 62, 36, 81,  2, 76, 37, 90],
       [56,  4, 50, 23, 28, 31, 81, 70],
       [ 1, 87, 65, 20, 52, 12, 95, 78],
       [ 1, 98, 77, 66, 43, 98, 55, 52]])

In [0]:
# Define the MIP solver

solver = pywraplp.Solver('Assignment-MIP',
                          pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


In [0]:
# Create the assignment matrix. Each key is a (worker, task) pair.

assign = {}

for w in range(num_workers):
  for t in range(num_tasks):
    assign[w, t] = solver.BoolVar('assign[%i,%i]' % (w, t))


In [0]:
# Objective Function

solver.Minimize(solver.Sum([cost_matrix[w][t] * assign[w,t] for w in range(num_workers)
                                                for t in range(num_tasks)]))


In [7]:
# Constraints

# Each worker is assigned to at most 1 task.

for w in range(num_workers):
  solver.Add(solver.Sum([assign[w, t] for t in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.

for t in range(num_tasks):
  solver.Add(solver.Sum([assign[w, t] for w in range(num_workers)]) == 1)

# Each team takes on the same amount of tasks.

solver.Add(solver.Sum([assign[w, t] for w in team_1 for t in range(num_tasks)]) <= num_tasks/num_teams)
solver.Add(solver.Sum([assign[w, t] for w in team_2 for t in range(num_tasks)]) <= num_tasks/num_teams)


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fe8c56c39c0> >

In [8]:
# Solve problem and report results

result = solver.Solve()

print("Elapsed time = ", solver.WallTime(), " ms")
print("Number of variables: ", solver.NumVariables())
print("Is feasible:", bool(solver.FEASIBLE))

if result == pywraplp.Solver.OPTIMAL:
    print('Optimal cost: ', solver.Objective().Value())
else:
    print('The problem does not have an optimal solution')

print('Total cost: ', solver.Objective().Value())
for w in range(num_workers):
  for t in range(num_tasks):
    if assign[w, t].solution_value() > 0:
      print('Worker %d --> Task %d @ Cost: %d' % (w, t, cost_matrix[w][t]))


Elapsed time =  72  ms
Number of variables:  80
Is feasible: True
Optimal cost:  78.0
Total cost:  78.0
Worker 0 --> Task 7 @ Cost: 11
Worker 1 --> Task 6 @ Cost: 13
Worker 2 --> Task 2 @ Cost: 18
Worker 3 --> Task 3 @ Cost: 17
Worker 6 --> Task 4 @ Cost: 2
Worker 7 --> Task 1 @ Cost: 4
Worker 8 --> Task 5 @ Cost: 12
Worker 9 --> Task 0 @ Cost: 1


### Model saving

In [0]:
def download_solver_as_mps(model):

  mps_text = model.ExportModelAsMpsFormat(True, False)

  created_time = datetime.now()
  filename = 'FSSP_{}.mps'.format(created_time)

  with open(filename, 'w') as file:
    file.write(mps_text)
    
  files.download(filename)

  return filename

In [0]:
def download_solver_as_lp(model):

  mps_text = model.ExportModelAsLpFormat(False)

  created_time = datetime.now()
  filename = 'FSSP_{}.lp'.format(created_time)

  with open(filename, 'w') as file:
    file.write(mps_text)
    
  files.download(filename)

  return filename